In [1]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib
from sobol_lib import *
from mpl_toolkits.mplot3d import Axes3D
from gen_mat import *
from block_rect_maxvol import *
import re
import os
from matplotlib import cm
from mva_test import *
import itertools
from ipywidgets import interactive, interact, widgets
#%matplotlib inline
%matplotlib notebook
import matplotlib.pyplot as plt
from maxvolpy.maxvol import rect_maxvol, maxvol
from numba import jit
os.environ['OMP_NUM_THREADS'] = '6'
print (os.environ['OMP_NUM_THREADS'])

6


In [2]:
num_expr = 1 # Adding this num to output fn for distinguish between experiments  (not used for now)
nder = 2 # Dimension
ToLoadSet = False # whether to load settigns form file

cut_radiuses = [0.05]
inital_points_distribs = ['LHS']

if ToLoadSet:
    from sandbox_bm_approx_settings import *
else:
    dir_str = './cr_test_blob'
    #dir_str = os.path.join(os.environ['HOME'], 'work/res/bm-big/')
    num_points_for_big_matrix = 6000 # number of points for big matrix
    max_row =  40                   # Maximum number of points taken in experiments
    max_expansion = 12         # number of columns in big matrix in (nder+1) units
    min_expansion = 10        # minimal number of columns in experiments in (nder+1) units
    

    n_test = 30000    # points on test grid (for calculating error on final step)
    poly = cheb       # used polinomials

    domain_type = 'blob'    #possible types: 'blob', 'ellipse', 'plane', 'rhombus'



#####
dir_pdf = os.path.join(dir_str, "pdf")
try:
    os.makedirs(dir_pdf)
except:
    pass



ToTakePointsFromFile = False # (not used for now)

# ---------------------------------
p_size = (nder+1)*max_row #number of rows in big matrix

### generating test points
points_test = complex_area_pnts_gen(n_test, nder, distrib='LHS', mod = domain_type)

#contents of file sandbox_bm_approx_settings.py
_="""
from gen_mat import cheb


dir_str = '.'
num_points_for_big_matrix = 50000 # number of points for big matrix
max_row = 50                      # Maximum number of points taken in experiments
max_expansion = 10                 # number of columns in big matrix in (nder+1) units
min_expansion = 1                 # minimal number of columns in experiments in (nder+1) units

n_test = 50000    # points on test grid (for calculating error on final step)
poly = cheb       # used polynomials
"""

### Testing the new most general environment

In [3]:
### evaluating test
for inital_points_distrib in inital_points_distribs:
    points_fn = 'taken_points_{}'.format(inital_points_distrib)
    x = complex_area_pnts_gen(num_points_for_big_matrix, nder, distrib='LHS', mod = domain_type)

    A = GenMat(p_size, x, poly=poly, debug=False, pow_p=1)
    A = matrix_prep(A, nder+1)
    
    np.savez(os.path.join(dir_str, points_fn), x=x, points_test=points_test)
    
    
    
    fn_pre_pdf = "distrib={}".format(inital_points_distrib)
    for cut_radius in cut_radiuses:
        f = open(os.path.join(dir_str, "distrib={}_radius={}".format(inital_points_distrib, cut_radius) + '.txt'), "w")
        for expansion in range(min_expansion, max_expansion):
                    for N_rows_ex in range(max_row, expansion, -1): # It's not the way people do...
                        N_rows = N_rows_ex*(nder+1)
                        fnpdf = os.path.join(dir_pdf, fn_pre_pdf + "_expansion={}_N_rows_ex={}.pdf".format(expansion, N_rows_ex))
                        try:
                            taken_points = test_bm(A, x,nder, expansion, N_rows, cut_radius = cut_radius,to_save_pivs=N_rows_ex==max_row, 
                                                       fnpdf=fnpdf)
                        except SingularError as err:
                            print ('not full column rank with expansion={}, N_rows_ex={}, err={}'.format(
                                                                expansion, N_rows_ex, err.value)) 
                            continue
                            


                        taken_points.tofile(f, sep=" ")
                        f.write("_Nrows={}_expans={}\n".format(N_rows, expansion))
                        f.flush()

        f.close()


### Experimental data processing

In [ ]:
### Data extraction
dir_str = './'
dir_points = os.path.join(dir_str, "cr_test_electron/")
points_distrib = 'LHS'
rad = "0.09"
taken_points = np.load(dir_points + "taken_points_" + points_distrib + ".npz")
x = taken_points['x']
points_test = taken_points['points_test']

In [ ]:
### .txt processing
functions = [f_roots]
N_row, N_col, p_indices = file_extraction(dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt")

In [ ]:
#error = np.empty(N_row.shape)
error_e, Leb_e = [], []
#error_sc, Leb_sc = [],[]
#Leb = np.empty(N_row.shape)

if type(functions) is not list:
        functions = [functions]
#if type(function2) is not list:
#        function2 = [function2]
ValsandNorms = MakeValsAndNorms(functions, points_test)
#ValsandNorms_sc = MakeValsAndNorms(function2, points_test)
for j, p in enumerate(p_indices):

    a,b = LebesgueConst(x[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
    #c,d = LebesgueConst(x[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms_sc, derivative = True)
    error_e.append(b)
    Leb_e.append(a)
    #error_sc.append(d)
    #Leb_sc.append(c)

In [ ]:
@jit
def mult_error(N_iter, N_col, p_indices,functions, shape=None):
    error_LHS = np.empty([N_col.shape[0], N_iter])
    error_random = np.empty([N_col.shape[0], N_iter])
    error_sobol = np.empty([N_col.shape[0], N_iter])
    if type(functions) is not list:
            functions = [functions]
    ValsandNorms = MakeValsAndNorms(functions, points_test)
    for i in np.arange(N_iter):
        print(i)
        x_LHS = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='LHS')
        x_random = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='random')
        x_sobol = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='Sobol')
        for j, p in enumerate(p_indices):
            _, error_LHS[j,i] = LebesgueConst(x_LHS, N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
            _, error_random[j,i] = LebesgueConst(x_random, N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
            _, error_sobol[j,i] = LebesgueConst(x_sobol, N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
            if j!= p_indices.shape[0]-1:
                if len(p_indices[j+1])>len(p_indices[j]):
                    x_LHS = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='LHS')
                    x_random = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='random')
                    x_sobol = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='Sobol')
                else:
                    x_LHS = x_LHS[:x_LHS.shape[0]-1]
                    x_random = x_random[:x_random.shape[0]-1]
                    x_sobol = x_sobol[:x_sobol.shape[0]-1]
    return(error_LHS,error_random,error_sobol)

In [ ]:
@jit
def multi_piston_error(N_iter, N_row, N_col, p_indices,functions):
    error_LHS = np.empty([N_row[::10].shape[0], N_iter])
    error_random = np.empty([N_row[::10].shape[0], N_iter])
    error_sobol = np.empty([N_row[::10].shape[0], N_iter])
    if type(functions) is not list:
            functions = [functions]
    for i in np.arange(N_iter):
        print(i)
        x_LHS = test_points_gen(num_points_for_big_matrix, nder, distrib='LHS')
        x_random = test_points_gen(num_points_for_big_matrix, nder, distrib='random')
        x_sobol = GenSobol( N = num_points_for_big_matrix, dim = nder, seed=random.randint(0,256),rng=(-1, 1) )
        for j, p in enumerate(p_indices):
            if (j%10 == 0):
                ValsandNorms = MakeValsAndNorms(functions, points_test)
                _, error_LHS[j/10,i] = LebesgueConst(x_LHS[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
                _, error_random[j/10,i] = LebesgueConst(x_random[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
                _, error_sobol[j/10,i] = LebesgueConst(x_sobol[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
                np.savez(os.path.join("piston_10"),er_lhs = error_LHS,er_r = error_random,er_s = error_sobol)

                return(error_LHS,error_random,error_sobol)

In [ ]:
a,b,c = mult_error(10,N_col,p_indices,functions)

In [ ]:
a,b,c = multi_piston_error(20,N_row,N_col,p_indices,functions)

In [ ]:
PlotError_3D(N_row, N_col, error_e, True)

### Random points plot

In [ ]:
def multi_error_straight(fn, error, err_LHS, err_rand, err_sob, col, rows, confidence = False):
    ndim = 3
    k = 0.403
    error = np.array(error)
    N_row, N_col,_ = file_extraction(fn, new_extr=True)
    error_LHS = np.mean(err_LHS, axis = 1) 
    error_random = np.mean(err_rand,axis = 1)
    error_sobol = np.mean(err_sob,axis = 1)
    if confidence:
        error_lhs_up = k*np.std(err_LHS,axis=1) + error_LHS
        error_lhs_down = error_LHS - k*np.std(err_LHS,axis=1)

        error_rand_up = k*np.std(err_rand,axis=1) + error_random
        error_rand_down = error_random - k*np.std(err_rand,axis=1)

        error_sob_up = k*np.std(err_sob,axis=1) + error_sobol
        error_sob_down = error_sobol - k*np.std(err_sob,axis=1)
  
    if col is not None:
        cols = np.arange(N_col[0], N_col[-1])
        #if type(cols) is not list:
            #cols = [cols]
        for i in cols:
            indx = np.where(N_col == i)          
            fig = plt.figure()
            plt.plot(N_row[indx]/ndim, error[indx], label = 'BMaxvol', linewidth= 3)
            plt.plot(N_row[indx]/ndim, error_LHS[indx], label = "LHS",color = '#539caf')
            plt.plot(N_row[indx]/ndim, error_random[indx], label = "Random",color = 'green')
            plt.plot(N_row[indx]/ndim, error_sobol[indx], label = "Sobol",color='red')
            if confidence:
                plt.fill_between(N_row[indx]/ndim, error_lhs_down[indx], error_lhs_up[indx],color = '#539caf',alpha = 0.4,label = '95% CI LHS')
                plt.fill_between(N_row[indx]/ndim, error_rand_down[indx], error_rand_up[indx],color = 'green',alpha = 0.4,label = '95% CI random')
                plt.fill_between(N_row[indx]/ndim, error_sob_down[indx], error_sob_up[indx],color = 'red', alpha = 0.4,   label = '95% CI Sobol')
            plt.yscale('log')
            plt.xlabel('Number of points', fontsize=10)
            plt.ylabel('$\log \epsilon$', rotation=90, labelpad=5)
            plt.legend()
            fnpdf = 'err(rows)_monoms={}.pdf'.format(ndim*i)
            plt.grid(True)
            plt.savefig(fnpdf)
            plt.close(fig)
           
    if rows is not None:
        if type(rows) is not list:
            rows = rows.tolist()
        for row in rows:
            indx = np.where(N_row == row)
            fig = plt.figure()
            plt.plot(ndim*N_col[indx], error[indx],label = 'BMaxvol', linewidth = 3)
            plt.plot(ndim*N_col[indx], error_LHS[indx], label = "LHS",color = '#539caf')
            plt.plot(ndim*N_col[indx], error_random[indx], label = "Random",color = 'green')
            plt.plot(ndim*N_col[indx], error_sobol[indx], label = "Sobol",color = 'red')
            if confidence:
                plt.fill_between(ndim*N_col[indx], error_lhs_down[indx], error_lhs_up[indx],color = '#539caf',alpha = 0.4,label = '95% CI LHS')
                plt.fill_between(ndim*N_col[indx], error_rand_down[indx], error_rand_up[indx],color = 'green',alpha = 0.4,label = '95% CI random')
                plt.fill_between(ndim*N_col[indx], error_sob_down[indx], error_sob_up[indx],  color = 'red',  alpha = 0.4,label = '95% CI Sobol')
            plt.yscale('log')
            plt.xlabel('Number of expansion terms', fontsize=10)
            plt.ylabel('$\log \epsilon$', rotation=90,labelpad=5)
            plt.legend()
            fnpdf = 'err(cols)_row={}.pdf'.format(row)
            plt.grid(True)
            plt.savefig(fnpdf)
            plt.close(fig)    
            
            
            
def only_bmaxvol_error(fn, error, rows):
    ndim = 3
    error = np.array(error)
    N_row, N_col,_ = file_extraction(fn, new_extr=True)
    if type(rows) is not list:
            rows = rows.tolist()
    for row in rows:
        indx = np.where(N_row == row)
        fig = plt.figure()
        plt.plot(ndim*N_col[indx], error[indx],label = 'BMaxvol', linewidth = 3)
        plt.yscale('log')
        plt.xlabel('Number of expansion terms', fontsize=10)
        plt.ylabel('$\log \epsilon$', rotation=90,labelpad=5)
        plt.legend()
        fnpdf = 'err(cols)_row={}.pdf'.format(row)
        plt.grid(True)
        plt.savefig(fnpdf)
        plt.close(fig)    


In [ ]:
only_bmaxvol_error(dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt", error_e,[210])

In [ ]:
multi_error_straight(dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt", error_e, a,b,c,1, np.arange(54,90,3),confidence=True)

### Draw areas

In [ ]:
#col_exp = min_expansion
col_exp = 40
i = np.where(N_col == col_exp)[0][0]
taken_indices = p_indices[i]
N_points = len(p_indices[i])

In [ ]:
parameter_plot(len(p_indices[i]))

In [ ]:
x = complex_area_pnts_gen(1400, nder, mod='plane', distrib='Sobol')

In [ ]:
plane_shape(x)

In [ ]:
def LebesgueConst_w(pnts, l,uwu, poly=cheb,test_pnts=None, pow_p=1, funcs=None):
    A = GenMat(l, pnts, poly=poly, debug=False, pow_p=pow_p, ToGenDiff=True)
    nder = pnts.shape[1]
    A = matrix_prep(A, nder+1)
    ewe = np.array([[1],[1],[1]])
    weeght = np.kron(uwu,ewe)
    C = weeght*A
    ABig = GenMat(l, test_pnts, poly=poly, debug=False, pow_p=pow_p, ToGenDiff=False)
    F = np.linalg.pinv(A).T.dot(ABig.T)
    for f, fvals, fnorm in funcs:
        rhs = RHS(f, pnts, derivative = True)
        print rhs.shape
        print weeght.shape
        rhs1 = weeght.T*rhs
        rhs1 = rhs1.reshape(rhs1.shape[1])
        #rhs = rhs1.reshape(42)
        res = np.linalg.norm(F.T.dot(rhs) - fvals, np.inf)/fnorm

    return res


In [ ]:
A = GenMat(6, x, poly=power, debug=False, pow_p=1)

In [ ]:
taken_points = np.load("piston_10_error.npz")

In [ ]:
x = taken_points['er']

In [ ]:
data = np.load("piston_10.npz")

In [ ]:
er_lhs = data['er_lhs']
er_r = data['er_r']
er_s = data['er_s']

In [ ]:
def multi_error_piston(error, err_LHS, err_rand, err_sob):
    ndim = 8
    k = 0.6
    error = np.array(error)
    error_LHS = np.mean(err_LHS, axis = 1) 
    error_random = np.mean(err_rand,axis = 1)
    error_sobol = np.mean(err_sob,axis = 1)

    error_lhs_up = k*np.std(err_LHS,axis=1) + error_LHS
    error_lhs_down = error_LHS - k*np.std(err_LHS,axis=1)

    error_rand_up = k*np.std(err_rand,axis=1) + error_random
    error_rand_down = error_random - k*np.std(err_rand,axis=1)

    error_sob_up = k*np.std(err_sob,axis=1) + error_sobol
    error_sob_down = error_sobol - k*np.std(err_sob,axis=1)
  

    indx = np.arange(0,10)
    N_row = np.arange(1600,800,-80)
    fig = plt.figure()
    plt.plot(N_row[indx]/ndim, error[indx], label = 'BMaxvol', linewidth= 3)
    plt.plot(N_row[indx]/ndim, error_LHS[indx], label = "LHS",color = '#539caf')
    plt.plot(N_row[indx]/ndim, error_random[indx], label = "Random",color = 'green')
    plt.plot(N_row[indx]/ndim, error_sobol[indx], label = "Sobol",color='red')
    
    
    plt.fill_between(N_row[indx]/ndim, error_lhs_down[indx], error_lhs_up[indx],color = '#539caf',alpha = 0.4,label = '95% CI LHS')
    plt.fill_between(N_row[indx]/ndim, error_rand_down[indx], error_rand_up[indx],color = 'green',alpha = 0.4,label = '95% CI random')
    plt.fill_between(N_row[indx]/ndim, error_sob_down[indx], error_sob_up[indx],color = 'red', alpha = 0.4,   label = '95% CI Sobol')
    
    plt.yscale('log')
    plt.xlabel('Number of points', fontsize=10)
    plt.ylabel('$\log \epsilon$', rotation=90, labelpad=5)
    plt.legend()
    fnpdf = 'err(rows)_monoms={}.pdf'.format(ndim*100)
    plt.grid(True)
    plt.savefig(fnpdf)
    plt.close(fig)

In [ ]:
multi_error_piston(x,er_lhs, er_r, er_s)

### Radial functions

In [ ]:
def ellipse_sp(x,y):
    b,e = 0.2,0.95
    sigma,n = 1/2, 1
    r = sp.sqrt(x**2 + y**2)
    phi = sp.atan2(y,x)
    R = r*sp.sqrt(1 - (e*sp.cos(phi))**2)/b
    return((sp.exp(-1*(R**2)/(2*(sigma**2))))*sp.cos(n*phi))

def sin_blob_sp(x,y):
    a = 0.2
    b = 0.8
    sigma,n = 1,7
    r = sp.sqrt(x**2 + y**2)
    phi = sp.atan2(y,x)
    R = r*(1/(b + a*sp.cos(n*phi)))
    return((sp.exp(-1*(R**2)/(2*(sigma**2))))*(r**2))

f_ellipse = symb_to_func(ellipse_sp,    2, True, False, name='Ellipse')  
f_sin_blob= symb_to_func(sin_blob_sp,   2, True, False, name='Blob')  

f_ellipse.diff    = MakeDiffs(ellipse_sp,  2)
f_sin_blob.diff   = MakeDiffs(sin_blob_sp, 2)

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.gca(projection='3d')

# Make data.
a = 1.1
X = np.linspace(-a, a, 1000)
Y = np.linspace(-a, a, 1000)
phi = np.arange(0,2*np.pi, 0.01)
b= 0.2
e = 0.95
X, Y = np.meshgrid(X, Y)

#r = (b / np.sqrt(1-(e*np.cos(phi))**2))
#xx = r*np.cos(phi)
#yy = r*np.sin(phi)

r = 0.8 + 0.2*np.cos(7*phi)
xx = r*np.cos(phi)
yy = r*np.sin(phi)

#Z = f_ellipse(X,Y)
#Z = f_sin_blob(X,Y)
Z = f_gauss_doubl(X,Y) - 2

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.summer,
                       linewidth=0,label = 'Trigonometric')
#fc = ax.plot(xx, yy,'bo', zs=0, zdir='z', label='curve in (x,y)')
# Customize the z axis.
ax.set_zlim(-1, 1)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.4, aspect=5)
plt.grid(True)
#fnpdf = 'ellips.pdf'
#plt.savefig(fnpdf)
plt.show()